<a href="https://colab.research.google.com/github/X4Zero/SISTEMAS_DE_RECOMENDACION/blob/master/SistemaRecomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SISTEMAS DE RECOMENDACIÓN
Primera semana en Hackspace, el primer proyecto que desarrollaré será un sistema de recomendación.  
[Enlace del tutorial](https://www.datacamp.com/community/tutorials/recommender-systems-python)

De acuerdo al tutorial hay 3 tipos de sistemas de recomendación, los cuales son:


*   Recomendadores simples: básicamente hablando de películas recomiendan aquellas con mejores resultados en críticas, pues tienen mayor probabilidad de gustarle al público promedio
*   Recomendadores basados en contenido: usan metadatos de items anteriores que han gustado a los usuarios, teniendo la idea de que a los usuarios les gustarán items similares a los que le han gustado antes.
*   Motores de filtrado colaborativo: predicen la preferencia o  puntuación de un usuario sobre un item en base a preferencias o puntuaciones de otros usuarios.




## DATASET
Contiene metadata de 45000 películas. Full MovieLens Dataset.

El dataset contiene:


*   movies_metadata.csv: This file contains information on ~45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, genre, revenue, release dates, languages, production countries, and companies.
*   keywords.csv: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.
*   credits.csv: Consists of Cast and Crew Information for all the movies. Available in the form of a stringified JSON Object.
*   links.csv: This file contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.
*   links_small.csv: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.
*   ratings_small.csv: The subset of 100,000 ratings from 700 users on 9,000 movies.


[Enlace del dataset completo](https://grouplens.org/datasets/movielens/latest/)

[Enlace del dataset usado para el trabajo](https://www.kaggle.com/rounakbanik/the-movies-dataset/data)

## RECOMENDADOR SIMPLE

In [ ]:
# Import Pandas
import pandas as pd
import matplotlib.pyplot as plt

# Load Movies Metadata
ruta_base = '/content/drive/My Drive/Colab Notebooks/HACKSPACE/SEMANA1/The_Movies_Dataset/'
metadata = pd.read_csv(ruta_base+'movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
registros, columnas = metadata.shape
print("{} registros".format(registros))
print("{} columnas".format(columnas))

45466 registros
24 columnas


Since you are trying to build a clone of IMDB's Top 250, let's use its weighted rating formula as a metric/score. Mathematically, it is represented as follows:

\begin{equation}
\text Weighted Rating (\bf WR) = \left({{\bf v} \over {\bf v} + {\bf m}} \cdot R\right) + \left({{\bf m} \over {\bf v} + {\bf m}} \cdot C\right)
\end{equation}

In the above equation,

v is the number of votes for the movie;

m is the minimum votes required to be listed in the chart;

R is the average rating of the movie;

C is the mean vote across the whole report.

You already have the values to v (vote_count) and R (vote_average) for each movie in the dataset. It is also possible to directly calculate C from this data.

In [ ]:
# Calculate mean of vote average column
# La calificación promedio para una película en IMDB es de 5.61 en una escala de 0 a 10
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [ ]:
# Calculate the minimum number of votes required to be in the chart, m
# el número de votos recibidos por la película en el percentil 90
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [ ]:
# estadísticas sobre la cantidad de votos
metadata['vote_count'].describe()

count    45460.000000
mean       109.897338
std        491.310374
min          0.000000
25%          3.000000
50%         10.000000
75%         34.000000
max      14075.000000
Name: vote_count, dtype: float64

In [ ]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [ ]:
metadata.shape

(45466, 24)

In [ ]:
# Calcular el puntaje para cada película dentro de las seleccionadas
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Podemos ver que hay varias coincidencias con los primeros puestos del top 250 de IMDB y el top 20 de películas resultado del recomendador simple

## RECOMENDADOR BASADO EN CONTENIDO

In [ ]:
#Print plot overviews of the first 5 movies.
#descripción de la trama de las películas
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:
metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45

In [ ]:
# subset de películas 
metadata_subset = metadata[0:25000].copy()

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata_subset['overview'] = metadata_subset['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata_subset['overview'])

#Output the shape of tfidf_matrix
print(tfidf_matrix.shape)

print("{} películas, {} palabras".format(tfidf_matrix.shape[0],tfidf_matrix.shape[1]))

(25000, 53130)
25000 películas, 53130 palabras


In [ ]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

['bertsolari',
 'berwick',
 'beryl',
 'berylune',
 'berzan',
 'berzano',
 'besa',
 'besco',
 'beseeches',
 'beseiged']

In [ ]:
tfidf_matrix.shape

(25000, 53130)

Calculamos el puntaje de similaridad usando 

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

(25000, 25000)

In [ ]:
cosine_sim[1]

array([0.01586868, 1.        , 0.04878944, ..., 0.01340316, 0.        ,
       0.        ])

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata_subset.index, index=metadata_subset['title']).drop_duplicates()

In [ ]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [ ]:
metadata_subset[metadata_subset['title'].str.contains('The Dark Knight',na=False)]['title']

12481                            The Dark Knight
18252                      The Dark Knight Rises
19792    Batman: The Dark Knight Returns, Part 1
20232    Batman: The Dark Knight Returns, Part 2
Name: title, dtype: object

Este sistema hace un buen trabajo encontrando películas con una descripción similar de la trama, aunque la calidad de estas recomendaciones no es tan buena."The Dark Knight Rises" nos retorna todas las películas de batman, mientras que es más probable que las personas a las que les gustaron la película se encuentren más inclinados a disfrutar otras películas del mismo director, Christopher Nolan

In [ ]:
get_recommendations('The Dark Knight')

18252                                The Dark Knight Rises
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
21194    Batman Unmasked: The Psychology of the Dark Kn...
20232              Batman: The Dark Knight Returns, Part 2
150                                         Batman Forever
19792              Batman: The Dark Knight Returns, Part 1
585                                                 Batman
18035                                     Batman: Year One
9230                    Batman Beyond: Return of the Joker
Name: title, dtype: object

In [ ]:
get_recommendations('The Godfather')

1178      The Godfather: Part II
1914     The Godfather: Part III
23126                 Blood Ties
11297           Household Saints
10821                   Election
17729          Short Sharp Shock
8653                Violent City
13177               I Am the Law
6977             Queen of Hearts
6711                    Mobsters
Name: title, dtype: object

In [ ]:
get_recommendations('Toy Story')

15348                    Toy Story 3
2997                     Toy Story 2
10301         The 40 Year Old Virgin
24523                      Small Fry
23843    Andy Hardy's Blonde Trouble
8327                       The Champ
1071           Rebel Without a Cause
11399         For Your Consideration
1932                       Condorman
21359       Andy Hardy's Double Life
Name: title, dtype: object

## RECOMENDADOR BASADO EN CRÉDITOS, GÉNEROS Y PALABRAS CLAVE

In [ ]:
# Load keywords and credits
credits = pd.read_csv(ruta_base + 'credits.csv')
keywords = pd.read_csv(ruta_base + 'keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [ ]:
# Print the first two movies of your newly merged metadata
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
# Import Numpy
import numpy as np

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]"
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]"


In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [ ]:
metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


In [ ]:
#usaremos solo una parte de las películas, por problemas con la memoria Ram
metadata_subset = metadata[:25000].copy()

In [ ]:
metadata_subset['soup']

0        jealousy toy boy tomhanks timallen donrickles ...
1        boardgame disappearance basedonchildren'sbook ...
2        fishing bestfriend duringcreditsstinger walter...
3        basedonnovel interracialrelationship singlemot...
4        baby midlifecrisis confidence stevemartin dian...
                               ...                        
24995    musical englishchannel swimmer estherwilliams ...
24996    hell basedonsong,poemorrhyme salvatorepapa art...
24997     tonyabatemarco scottadsit mattbesser seanmere...
24998    prison bankrobbery fingerprints sidneytoler ma...
24999                     davidspade keithtruesdell comedy
Name: soup, Length: 25000, dtype: object

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(metadata['soup'])
count_matrix = count.fit_transform(metadata_subset['soup'])

In [ ]:
count_matrix.shape

(25000, 44442)

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your main DataFrame and construct reverse mapping as before
# metadata = metadata.reset_index()
# indices = pd.Series(metadata.index, index=metadata['title'])
metadata_subset = metadata_subset.reset_index()
indices = pd.Series(metadata_subset.index, index=metadata_subset['title'])

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations_sub(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata_subset['title'].iloc[movie_indices]

In [ ]:
get_recommendations_sub('The Dark Knight Rises', cosine_sim2)

12589      The Dark Knight
10210        Batman Begins
9311                Shiner
9874       Amongst Friends
7772              Mitchell
516      Romeo Is Bleeding
11463         The Prestige
24090            Quicksand
10853       Helter Skelter
18940            Last Exit
Name: title, dtype: object

In [ ]:
get_recommendations_sub('The Godfather', cosine_sim2)

In [ ]:
get_recommendations_sub('Toy Story', cosine_sim2)

In [ ]:
get_recommendations_sub('Superman', cosine_sim2)

In [ ]:
metadata_subset[metadata_subset['title'].str.contains('Superman',na=False)]['title']

## Preparación del Sistema Recomendador Basado en contenido para pasar a una app web en flask

In [ ]:
# Import Pandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load Movies Metadata
ruta_base = '/content/drive/My Drive/Colab Notebooks/HACKSPACE/SEMANA1/The_Movies_Dataset/'
metadata_original = pd.read_csv(ruta_base+'movies_metadata.csv', low_memory=False)

# Load keywords and credits
credits = pd.read_csv(ruta_base + 'credits.csv')
keywords = pd.read_csv(ruta_base + 'keywords.csv')

# Remove rows with bad IDs.
metadata_original = metadata_original.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata_original['id'] = metadata_original['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata_original = metadata_original.merge(credits, on='id')
metadata_original = metadata_original.merge(keywords, on='id')

In [ ]:
########################
# FUNCIONES NECESARIAS #
########################

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

def get_list_value(x,col_value):
    if isinstance(x, list):
        names = [i[col_value] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        # if len(names) > 3:
        #     names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
  

def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


# Function that takes in movie title as input and outputs most similar movies
def obtener_recomendaciones(title, cosine_sim,indices,df):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [ ]:
############################
# PREPARACION DE LOS DATOS #
############################

# Parse the stringified features into their corresponding python objects
from ast import literal_eval

def preparar_datos(df_metadata):
  features = ['cast', 'crew', 'keywords', 'genres']
  for feature in features:
      df_metadata[feature] = df_metadata[feature].apply(literal_eval)


  # Define new director, cast, genres and keywords features that are in a suitable form.
  df_metadata['director'] = df_metadata['crew'].apply(get_director)

  features = ['cast', 'keywords', 'genres']
  for feature in features:
      df_metadata[feature] = df_metadata[feature].apply(get_list)


  # Apply clean_data function to your features.
  features = ['cast', 'keywords', 'director', 'genres']

  for feature in features:
      df_metadata[feature] = df_metadata[feature].apply(clean_data)


  # Create a new soup feature
  df_metadata['soup'] = df_metadata.apply(create_soup, axis=1)

  return df_metadata

Por problemas de memoria, no se trabajará con las 45000 películas, sino se utilizará una parte de estos datos

In [ ]:
# Número de películas en total
print('Número de Películas: {}'.format(metadata_original.shape[0]))

metadata_original = metadata_original.drop_duplicates(subset=['title'])
metadata_original['release_date'] = pd.to_datetime(metadata_original['release_date'])
metadata_prod = metadata_original[metadata_original['release_date']>'1999-05-18']
numero_peliculas = metadata_prod.shape[0]
print('Número de Películas después de 1999: {}'.format(numero_peliculas))
metadata_prod = metadata_prod.reset_index()
#Aún ocupan 

Número de Películas: 46628
Número de Películas después de 1999: 22605


In [ ]:
metadata_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22605 entries, 0 to 22604
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index                  22605 non-null  int64         
 1   adult                  22605 non-null  object        
 2   belongs_to_collection  2211 non-null   object        
 3   budget                 22605 non-null  object        
 4   genres                 22605 non-null  object        
 5   homepage               6751 non-null   object        
 6   id                     22605 non-null  int64         
 7   imdb_id                22597 non-null  object        
 8   original_language      22599 non-null  object        
 9   original_title         22605 non-null  object        
 10  overview               22084 non-null  object        
 11  popularity             22605 non-null  object        
 12  poster_path            22422 non-null  object        
 13  p

### Exploración de los lenguajes

In [ ]:
metadata_prod['spoken_languages']

0        [{'iso_639_1': 'it', 'name': 'Italiano'}]
1          [{'iso_639_1': 'pl', 'name': 'Polski'}]
2                                               []
3         [{'iso_639_1': 'en', 'name': 'English'}]
4         [{'iso_639_1': 'en', 'name': 'English'}]
                           ...                    
22600     [{'iso_639_1': 'en', 'name': 'English'}]
22601     [{'iso_639_1': 'en', 'name': 'English'}]
22602     [{'iso_639_1': 'en', 'name': 'English'}]
22603            [{'iso_639_1': 'tl', 'name': ''}]
22604     [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 22605, dtype: object

In [ ]:
# print('Data de películas - lenguajes:\n{}'.format(metadata_prod['spoken_languages'].to_string()))

In [ ]:
metadata_prod.shape

(22605, 28)

In [ ]:
ruta_lenguajes = '/'.join(ruta_base.split('/')[:-2])+'/languages.csv'
df_lenguajes = pd.read_csv(ruta_lenguajes)
df_lenguajes.head(5)

,ISO language name,Native name (endonym),639-1
0,Abkhazian,"аҧсуа бызшәа, аҧсшәа",ab
1,Afar,Afaraf,aa
2,Afrikaans,Afrikaans,af
3,Akan,Akan,ak
4,Albanian,Shqip,sq


In [ ]:
print('Aún hay {} películas en el dataset'.format(metadata_prod.shape[0]))


Aún hay 22605 películas en el dataset


In [ ]:
metadata_prod['lang'] = metadata_prod['spoken_languages'].apply(literal_eval)

In [ ]:
# obtenemos los codigos iso de los lenguajes
liso = metadata_prod['lang'].apply(lambda x: get_list_value(x,'iso_639_1')).values.tolist()
df_liso = pd.DataFrame(liso)

# filtramos para tener los códigos iso de los lenguajes sin que se repitan
lista_lenguajes = []

for col in df_liso.columns:
  lista_lenguajes += list(df_liso[col].unique())

set_lenguajes = set(lista_lenguajes)

codigos_lenguajes = list(set_lenguajes)
codigos_lenguajes.remove(None)

cantidad_codigos_lenguajes = len(codigos_lenguajes)
print("Hay {} códigos iso de lenguajes".format(cantidad_codigos_lenguajes))

Hay 125 códigos iso de lenguajes


In [ ]:
codigos_lenguajes

['be',
 'nl',
 'to',
 'ja',
 'fa',
 'th',
 'ps',
 'el',
 'hu',
 'nb',
 'rw',
 'so',
 'lt',
 'km',
 'st',
 'id',
 'mk',
 'ur',
 'fr',
 'et',
 'sg',
 'sw',
 'my',
 'bg',
 'dz',
 'ru',
 'pa',
 'ka',
 'es',
 'mn',
 'gu',
 'co',
 'cr',
 'am',
 'ab',
 'is',
 'ln',
 'no',
 'ca',
 'tk',
 'ga',
 'bn',
 'xx',
 'de',
 'ne',
 'jv',
 'sl',
 'gd',
 'tt',
 'da',
 'sq',
 'kk',
 'sh',
 'lv',
 'hi',
 'bo',
 'te',
 'pt',
 'hr',
 'ig',
 'ar',
 'lb',
 'zu',
 'ko',
 'sc',
 'yi',
 'uk',
 'tn',
 'ay',
 'tg',
 'ki',
 'vi',
 'hy',
 'ha',
 'pl',
 'af',
 'ms',
 'sr',
 'tl',
 'it',
 'gl',
 'sm',
 'br',
 'eu',
 'si',
 'gn',
 'bm',
 'fo',
 'zh',
 'fy',
 'sk',
 'se',
 'la',
 'he',
 'bi',
 'cn',
 'iu',
 'nv',
 'en',
 'lo',
 'fi',
 'ro',
 'eo',
 'uz',
 'cy',
 'ny',
 'wo',
 'ta',
 'sa',
 'sv',
 'cs',
 'ku',
 'kn',
 'tr',
 'ky',
 'ht',
 'qu',
 'mi',
 'mt',
 'oc',
 'mr',
 'xh',
 'ml',
 'ce',
 'bs']

In [ ]:
df_lenguajes = df_lenguajes.set_index('639-1').iloc[:]
df_lenguajes.columns = ['ISO language name','Native name']
diccionario_lenguajes = df_lenguajes.to_dict('index')
# es necesario agregar xx
diccionario_lenguajes['xx'] = {'ISO language name':'No Language','Native name':''}
diccionario_lenguajes['cn'] = {'ISO language name':'Cantonés estándar','Native name':'广州话 / 廣州話'}
diccionario_lenguajes['sh'] = {'ISO language name':'No Language','Native name':''}
diccionario_lenguajes

{'aa': {'ISO language name': 'Afar', 'Native name': 'Afaraf'},
 'ab': {'ISO language name': 'Abkhazian',
  'Native name': 'аҧсуа бызшәа, аҧсшәа'},
 'ae': {'ISO language name': 'Avestan', 'Native name': 'avesta'},
 'af': {'ISO language name': 'Afrikaans', 'Native name': 'Afrikaans'},
 'ak': {'ISO language name': 'Akan', 'Native name': 'Akan'},
 'am': {'ISO language name': 'Amharic', 'Native name': 'አማርኛ'},
 'an': {'ISO language name': 'Aragonese', 'Native name': 'aragonés'},
 'ar': {'ISO language name': 'Arabic', 'Native name': 'العربية'},
 'as': {'ISO language name': 'Assamese', 'Native name': 'অসমীয়া'},
 'av': {'ISO language name': 'Avaric',
  'Native name': 'авар мацӀ, магӀарул мацӀ'},
 'ay': {'ISO language name': 'Aymara', 'Native name': 'aymar aru'},
 'az': {'ISO language name': 'Azerbaijani', 'Native name': 'azərbaycan dili'},
 'ba': {'ISO language name': 'Bashkir', 'Native name': 'башҡорт теле'},
 'be': {'ISO language name': 'Belarusian', 'Native name': 'беларуская мова'},
 'bg'

In [ ]:
df_liso

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,it,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,pl,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,en,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,en,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22600,en,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
22601,en,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
22602,en,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
22603,tl,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
df_liso['posicion'] = df_liso.index.values

In [ ]:
df_liso_pro = df_liso.melt(id_vars='posicion').drop('variable',1)
df_liso_pro

,posicion,value
0,0,it
1,1,pl
2,2,None
3,3,en
4,4,en
...,...,...
429490,22600,None
429491,22601,None
429492,22602,None
429493,22603,None


In [ ]:
df_liso_pro[df_liso_pro['posicion']==22604]

,posicion,value
22604,22604,en
45209,22604,None
67814,22604,None
90419,22604,None
113024,22604,None
135629,22604,None
158234,22604,None
180839,22604,None
203444,22604,None
226049,22604,None


In [ ]:
posicion_v_lenguaje = pd.crosstab(df_liso_pro['posicion'],df_liso_pro['value'])
posicion_v_lenguaje

value,ab,af,am,ar,ay,be,bg,bi,bm,bn,bo,br,bs,ca,ce,cn,co,cr,cs,cy,da,de,dz,el,en,eo,es,et,eu,fa,fi,fo,fr,fy,ga,gd,gl,gn,gu,ha,...,pt,qu,ro,ru,rw,sa,sc,se,sg,sh,si,sk,sl,sm,so,sq,sr,st,sv,sw,ta,te,tg,th,tk,tl,tn,to,tr,tt,uk,ur,uz,vi,wo,xh,xx,yi,zh,zu
posicion,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22601,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# saber que en que lenguajes se encuentran las películas del dataset
for codigo in codigos_lenguajes:
  print('codigo: {}, ISO language name: {}'.format(codigo,diccionario_lenguajes[codigo]['ISO language name']))

codigo: be, ISO language name: Belarusian
codigo: nl, ISO language name: Dutch, Flemish
codigo: to, ISO language name: Tonga (Tonga Islands)
codigo: ja, ISO language name: Japanese
codigo: fa, ISO language name: Persian
codigo: th, ISO language name: Thai
codigo: ps, ISO language name: Pashto, Pushto
codigo: el, ISO language name: Greek, Modern (1453–)
codigo: hu, ISO language name: Hungarian
codigo: nb, ISO language name: Norwegian Bokmål
codigo: rw, ISO language name: Kinyarwanda
codigo: so, ISO language name: Somali
codigo: lt, ISO language name: Lithuanian
codigo: km, ISO language name: Central Khmer
codigo: st, ISO language name: Southern Sotho
codigo: id, ISO language name: Indonesian
codigo: mk, ISO language name: Macedonian
codigo: ur, ISO language name: Urdu
codigo: fr, ISO language name: French
codigo: et, ISO language name: Estonian
codigo: sg, ISO language name: Sango
codigo: sw, ISO language name: Swahili
codigo: my, ISO language name: Burmese
codigo: bg, ISO language name

In [ ]:
#codigos de los que se permiten todas las peliculas
codigos_permitidos = ['es','en','fr','ja','ko','qu','ay','it']

print("cantidad de codigos de lenguajes en el dataset: {}".format(len(codigos_lenguajes)))

codigos_no_permitidos = set(codigos_lenguajes) - set(codigos_permitidos)
print("cantidad de codigos de lenguaje de las películas que se removerán si solo tienen un lenguaje: {}".format(len(codigos_no_permitidos)))

cantidad de codigos de lenguajes en el dataset: 125
cantidad de codigos de lenguaje de las películas que se removerán si solo tienen un lenguaje: 117


In [ ]:
lista_indices_quitar=[]
for codigo in codigos_no_permitidos:
  x = posicion_v_lenguaje[posicion_v_lenguaje[codigo]==1]
  indices = list(x[x.sum(1) == 1].index)
  lista_indices_quitar+=indices

# cantidad de peliculas que restaremos al dataset
print("cantidad de películas que restaremos al dataset: {}".format(len(lista_indices_quitar)))

cantidad de películas que restaremos al dataset: 3012


In [ ]:
# dataset filtrado por lenguajes
metadata_prod_final =  metadata_prod[~metadata_prod.index.isin(lista_indices_quitar)]
metadata_prod_final = metadata_prod_final.reset_index(drop=True)
metadata_prod_final.head()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,lang
0,714,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,131232,tt0333373,en,Due Amici,"Two Sicilian friends, Nunzio and Pino, share t...",0.003949,/jo4n8M8EHedIuatB39C8EEmJBH8.jpg,[],"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2002-03-20,0.0,86.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Two Friends,False,0.0,0.0,"[{'cast_id': 5, 'character': 'Nunzio', 'credit...","[{'credit_id': '52fe4b72c3a368484e1896cd', 'de...",[],"[{'iso_639_1': 'it', 'name': 'Italiano'}]"
1,1102,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,[],[],2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[],[]
2,1675,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,216794,tt0119655,en,Men of Means,In a world where two men in close proximity wi...,0.022078,/c78yM65QpGBONxVEMBsTSuz098G.jpg,"[{'name': 'Saban Entertainment', 'id': 2106}, ...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",1999-05-25,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Double deal. Double cross. The only thing left...,Men of Means,False,0.0,0.0,"[{'cast_id': 2, 'character': 'Rico ""Bullet"" Bu...","[{'credit_id': '52fe4e20c3a368484e20fb7f', 'de...","[{'id': 10391, 'name': 'mafia'}, {'id': 11578,...","[{'iso_639_1': 'en', 'name': 'English'}]"
3,2134,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,/ePPNVWyIKYBdsGrOrYVaPKM8DlM.jpg,[],[],2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0,"[{'cast_id': 10, 'character': 'The Auctioneer'...","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...","[{'id': 214549, 'name': 'short'}]","[{'iso_639_1': 'en', 'name': 'English'}]"
4,2144,False,NaN,0,"[{'id': 37, 'name': 'Western'}, {'id': 28, 'na...",NaN,69848,tt0120775,en,One Man's Hero,One Man's Hero tells the little-known story of...,0.996778,/vgvVNmCybkmOtaSXpSG7aONQrJ2.jpg,"[{'name': 'Filmax', 'id': 3631}, {'name': 'Sil...","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-08-02,0.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One man's hero is another man's traitor.,One Man's Hero,False,9.3,2.0,"[{'cast_id': 0, 'character': 'Captain John Ril...","[{'credit_id': '558603f4c3a36837e90007f2', 'de...","[{'id': 6091, 'name': 'war'}, {'id': 6092, 'na...","[{'iso_639_1': 'en', 'name': 'English'}]"


In [ ]:
#peliculas quitadas del dataset original
metadata_prod.loc[lista_indices_quitar]['title']

2753                     Yesterday
679            Everybody's Famous!
767             Pauline & Paulette
2142        The Memory Of A Killer
4343                        Waiter
                   ...            
18803                  From A to B
18977    Sorry for the Disturbance
19153            Dawn of the World
19348              Eyes of a Thief
21077                   Sand Storm
Name: title, Length: 3012, dtype: object

In [ ]:
metadata_prod.loc[2142]

index                                                                 9547
adult                                                                False
belongs_to_collection                                                  NaN
budget                                                                   0
genres                   [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
homepage                      http://www.sonyclassics.com/memoryofakiller/
id                                                                   24860
imdb_id                                                          tt0374345
original_language                                                       nl
original_title                                           De Zaak Alzheimer
overview                 Vincke and Verstuyft are one of the best detec...
popularity                                                        4.368257
poster_path                               /2COdg7as2jbEm5WiSlmXcaqoPfz.jpg
production_companies     

In [ ]:
#peliculas en el lenguaje Telugu solo con ese lenguaje
posicion_v_lenguaje.loc[(posicion_v_lenguaje[posicion_v_lenguaje['te']==1].sum(1) == 1).index]

value,ab,af,am,ar,ay,be,bg,bi,bm,bn,bo,br,bs,ca,ce,cn,co,cr,cs,cy,da,de,dz,el,en,eo,es,et,eu,fa,fi,fo,fr,fy,ga,gd,gl,gn,gu,ha,...,pt,qu,ro,ru,rw,sa,sc,se,sg,sh,si,sk,sl,sm,so,sq,sr,st,sv,sw,ta,te,tg,th,tk,tl,tn,to,tr,tt,uk,ur,uz,vi,wo,xh,xx,yi,zh,zu
posicion,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8386,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8568,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21267,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21568,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
metadata_prod_final.head(3)

,level_0,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,lang
0,0,714,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,131232,tt0333373,en,Due Amici,"Two Sicilian friends, Nunzio and Pino, share t...",0.003949,/jo4n8M8EHedIuatB39C8EEmJBH8.jpg,[],"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2002-03-20,0.0,86.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Two Friends,False,0.0,0.0,"[{'cast_id': 5, 'character': 'Nunzio', 'credit...","[{'credit_id': '52fe4b72c3a368484e1896cd', 'de...",[],"[{'iso_639_1': 'it', 'name': 'Italiano'}]"
1,2,1102,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,[],[],2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0,"[{'cast_id': 2, 'character': 'Rachel', 'credit...","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[],[]
2,3,1675,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,216794,tt0119655,en,Men of Means,In a world where two men in close proximity wi...,0.022078,/c78yM65QpGBONxVEMBsTSuz098G.jpg,"[{'name': 'Saban Entertainment', 'id': 2106}, ...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",1999-05-25,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Double deal. Double cross. The only thing left...,Men of Means,False,0.0,0.0,"[{'cast_id': 2, 'character': 'Rico ""Bullet"" Bu...","[{'credit_id': '52fe4e20c3a368484e20fb7f', 'de...","[{'id': 10391, 'name': 'mafia'}, {'id': 11578,...","[{'iso_639_1': 'en', 'name': 'English'}]"


In [ ]:
#preparamos los  datos
preparar_datos(metadata_prod_final)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,lang,director,soup
0,714,False,NaN,0,"[drama, foreign]",NaN,131232,tt0333373,en,Due Amici,"Two Sicilian friends, Nunzio and Pino, share t...",0.003949,/jo4n8M8EHedIuatB39C8EEmJBH8.jpg,[],"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2002-03-20,0.0,86.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Two Friends,False,0.0,0.0,"[francescosframeli, spiroscimone]","[{'credit_id': '52fe4b72c3a368484e1896cd', 'de...",[],"[{'iso_639_1': 'it', 'name': 'Italiano'}]",francescosframeli,francescosframeli spiroscimone francescosfram...
1,1102,False,NaN,0,"[comedy, horror]",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,[],[],2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0,"[joshfeldman, guskamp, carolynjania]","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[],[],chriscullari,joshfeldman guskamp carolynjania chriscullari...
2,1675,False,NaN,0,"[drama, action]",NaN,216794,tt0119655,en,Men of Means,In a world where two men in close proximity wi...,0.022078,/c78yM65QpGBONxVEMBsTSuz098G.jpg,"[{'name': 'Saban Entertainment', 'id': 2106}, ...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",1999-05-25,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Double deal. Double cross. The only thing left...,Men of Means,False,0.0,0.0,"[michaelparé, kaeladobkin, austinpendleton]","[{'credit_id': '52fe4e20c3a368484e20fb7f', 'de...","[mafia, mobster, brooklynnewyorkcity]","[{'iso_639_1': 'en', 'name': 'English'}]",georgemendeluk,mafia mobster brooklynnewyorkcity michaelparé ...
3,2134,False,NaN,0,[drama],http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,/ePPNVWyIKYBdsGrOrYVaPKM8DlM.jpg,[],[],2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0,"[jamescartwright, geraldinejames, alexkelly]","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...",[short],"[{'iso_639_1': 'en', 'name': 'English'}]",francislee,short jamescartwright geraldinejames alexkelly...
4,2144,False,NaN,0,"[western, action, drama]",NaN,69848,tt0120775,en,One Man's Hero,One Man's Hero tells the little-known story of...,0.996778,/vgvVNmCybkmOtaSXpSG7aONQrJ2.jpg,"[{'name': 'Filmax', 'id': 3631}, {'name': 'Sil...","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-08-02,0.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One man's hero is another man's traitor.,One Man's Hero,False,9.3,2.0,"[tomberenger, joaquimdealmeida, danielaromo]","[{'credit_id': '558603f4c3a36837e90007f2', 'de...","[war, army, battlefield]","[{'iso_639_1': 'en', 'name': 'English'}]",lancehool,war army battlefield tomberenger joaquimdealme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19588,46603,False,NaN,0,[comedy],NaN,14008,tt0294425,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,4.392389,/9sQVYo7B9cSexWXoQVzFFijCbD2.jpg,"[{'name': 'Walt Disney Television', 'id': 670}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2002-03-07,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Too Cool For The Rules!,Cadet Kelly,False,5.2,145.0,"[hilaryduff, christycarlsonromano, garycole]","[{'credit_id': '52fe45c29251416c75061803', 'de...",[militaryschool],"[{'iso_639_1': 'en', 'name': 'English'}]",larryshaw,militaryschool hilaryduff christycarlsonromano...
19589,46611,False,NaN,0,"[animation, family]",NaN,14885,tt0457437,en,Pooh's Heffalump Halloween Movie,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,/yU6byKhpzHG1RFpY5PYM6mfhfU1.jpg,"[{'name': 'Wa

In [ ]:


# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(metadata['soup'])
count_matrix = count.fit_transform(metadata_prod_final['soup'])


# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


metadata_prod_final = metadata_prod_final.reset_index(drop=True)
indices = pd.Series(metadata_prod_final.index, index=metadata_prod_final['title'])

In [ ]:
def ObtenerPeliculaCoincidencia(titulo):
  titulo=titulo.lower()
  slice_df = metadata_prod_final[metadata_prod_final['title'].str.lower().str.contains(titulo)]['title']
  return slice_df

def ObtenerRecomendacion(idx,df,cosine_sim,movie_indices):
  # Get the pairwsie similarity scores of all movies with that movie
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Get the scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]

  # Get the movie indices
  movie_indices = [i[0] for i in sim_scores]

  # Return the top 10 most similar movies
  return df['title'].iloc[movie_indices]

In [ ]:
ObtenerPeliculaCoincidencia('dragon ball')

8777                      Dragon Ball Z: Battle of Gods
12300    Dragon Ball: Yo! Son Goku and Friends Return!!
12707                   Dragon Ball Z: Resurrection 'F'
12915                   Dragon Ball: Episode of Bardock
Name: title, dtype: object

In [ ]:
ObtenerRecomendacion(12707,metadata_prod_final,cosine_sim2,indices)

12915                      Dragon Ball: Episode of Bardock
12300       Dragon Ball: Yo! Son Goku and Friends Return!!
3930                                             Dead Fury
8794                                       Pokémon Origins
2011     Inuyasha the Movie 2: The Castle Beyond the Lo...
7118                       Bleach the Movie: Fade to Black
8579                                      One Piece Film Z
19050                    Bleach: The DiamondDust Rebellion
11309                                     Chaar Sahibzaade
4967              Spiderman: The Ultimate Villain Showdown
Name: title, dtype: object

In [ ]:
cosine_sim2.shape

(19593, 19593)

In [ ]:
cosine_sim2[:5000,:5000].nbytes/1024/1024/1024

0.1862645149230957

In [ ]:
indices.nbytes/1024/1024

0.14948272705078125

In [ ]:
metadata_prod[metadata_prod['title'].str.contains('Star War')]['title']

841           Star Wars: Episode II - Attack of the Clones
2488          Star Wars: Episode III - Revenge of the Sith
4343                             Star Wars: The Clone Wars
5919     Empire of Dreams: The Story of the Star Wars T...
12649                         Star Wars: The Force Awakens
14854                             Robot Chicken: Star Wars
18356          Plastic Galaxy: The Story of Star Wars Toys
18357          Plastic Galaxy: The Story of Star Wars Toys
22478                         Rogue One: A Star Wars Story
Name: title, dtype: object

In [ ]:
metadata_prod_final[metadata_prod_final['title'].str.lower().str.contains('star wars',na=False)]['title']

5                Star Wars: Episode I - The Phantom Menace
799           Star Wars: Episode II - Attack of the Clones
2267          Star Wars: Episode III - Revenge of the Sith
3874                             Star Wars: The Clone Wars
5168     Empire of Dreams: The Story of the Star Wars T...
10706                         Star Wars: The Force Awakens
12423                             Robot Chicken: Star Wars
14891          Plastic Galaxy: The Story of Star Wars Toys
17611                         Rogue One: A Star Wars Story
Name: title, dtype: object

In [ ]:
metadata_original.loc[5]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                            60000000
genres                   [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
homepage                                                               NaN
id                                                                     949
imdb_id                                                          tt0113277
original_language                                                       en
original_title                                                        Heat
overview                 Obsessive master thief, Neil McCauley leads a ...
popularity                                                       17.924927
poster_path                               /zMyfPUelumio3tiDKPffaUpsQTD.jpg
production_companies     [{'name': 'Regency Enterprises', 'id': 508}, {...
production_countries     

In [ ]:
ruta_carpeta = '/'.join(ruta_base.split('/')[:-2])
metadata_prod_final.to_csv(ruta_carpeta+'/metadata_prod_final.csv',index=False)

In [ ]:
metadata_prod_final

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,lang,director,soup
0,714,False,NaN,0,"[drama, foreign]",NaN,131232,tt0333373,en,Due Amici,"Two Sicilian friends, Nunzio and Pino, share t...",0.003949,/jo4n8M8EHedIuatB39C8EEmJBH8.jpg,[],"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2002-03-20,0.0,86.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Two Friends,False,0.0,0.0,"[francescosframeli, spiroscimone]","[{'credit_id': '52fe4b72c3a368484e1896cd', 'de...",[],"[{'iso_639_1': 'it', 'name': 'Italiano'}]",francescosframeli,francescosframeli spiroscimone francescosfram...
1,1102,False,NaN,0,"[comedy, horror]",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",0.135596,/pQpyEmFTGaox4yLuUXmiD2IDIbA.jpg,[],[],2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0,"[joshfeldman, guskamp, carolynjania]","[{'credit_id': '52fe4aaf9251416c750ea6f1', 'de...",[],[],chriscullari,joshfeldman guskamp carolynjania chriscullari...
2,1675,False,NaN,0,"[drama, action]",NaN,216794,tt0119655,en,Men of Means,In a world where two men in close proximity wi...,0.022078,/c78yM65QpGBONxVEMBsTSuz098G.jpg,"[{'name': 'Saban Entertainment', 'id': 2106}, ...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",1999-05-25,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Double deal. Double cross. The only thing left...,Men of Means,False,0.0,0.0,"[michaelparé, kaeladobkin, austinpendleton]","[{'credit_id': '52fe4e20c3a368484e20fb7f', 'de...","[mafia, mobster, brooklynnewyorkcity]","[{'iso_639_1': 'en', 'name': 'English'}]",georgemendeluk,mafia mobster brooklynnewyorkcity michaelparé ...
3,2134,False,NaN,0,[drama],http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",0.211754,/ePPNVWyIKYBdsGrOrYVaPKM8DlM.jpg,[],[],2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0,"[jamescartwright, geraldinejames, alexkelly]","[{'credit_id': '52fe4b169251416c750f7cd5', 'de...",[short],"[{'iso_639_1': 'en', 'name': 'English'}]",francislee,short jamescartwright geraldinejames alexkelly...
4,2144,False,NaN,0,"[western, action, drama]",NaN,69848,tt0120775,en,One Man's Hero,One Man's Hero tells the little-known story of...,0.996778,/vgvVNmCybkmOtaSXpSG7aONQrJ2.jpg,"[{'name': 'Filmax', 'id': 3631}, {'name': 'Sil...","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-08-02,0.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One man's hero is another man's traitor.,One Man's Hero,False,9.3,2.0,"[tomberenger, joaquimdealmeida, danielaromo]","[{'credit_id': '558603f4c3a36837e90007f2', 'de...","[war, army, battlefield]","[{'iso_639_1': 'en', 'name': 'English'}]",lancehool,war army battlefield tomberenger joaquimdealme...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19588,46603,False,NaN,0,[comedy],NaN,14008,tt0294425,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,4.392389,/9sQVYo7B9cSexWXoQVzFFijCbD2.jpg,"[{'name': 'Walt Disney Television', 'id': 670}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2002-03-07,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Too Cool For The Rules!,Cadet Kelly,False,5.2,145.0,"[hilaryduff, christycarlsonromano, garycole]","[{'credit_id': '52fe45c29251416c75061803', 'de...",[militaryschool],"[{'iso_639_1': 'en', 'name': 'English'}]",larryshaw,militaryschool hilaryduff christycarlsonromano...
19589,46611,False,NaN,0,"[animation, family]",NaN,14885,tt0457437,en,Pooh's Heffalump Halloween Movie,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,/yU6byKhpzHG1RFpY5PYM6mfhfU1.jpg,"[{'name': 'Wa